# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments              Customer  \
0  Oct  6 2022 10:14AM  248037        15  PNC391408A  Person & Covey, Inc.   
1  Oct  6 2022 10:14AM  248037        15  PNC391412A  Person & Covey, Inc.   
2  Oct  6 2022 10:14AM  248037        15  PNC391414A  Person & Covey, Inc.   
3  Oct  6 2022 10:14AM  248037        15  PNC391415A  Person & Covey, Inc.   
4  Oct  6 2022 10:14AM  248037        15  PNC391418A  Person & Covey, Inc.   
5  Oct  6 2022 10:14AM  248037        15  PNC391419A  Person & Covey, Inc.   
6  Oct  6 2022 10:14AM  248037        15  PNC391422A  Person & Covey, Inc.   
7  Oct  6 2022 10:14AM  248037        15  PNC394722A  Person & Covey, Inc.   
8  Oct  6 2022 10:11AM  248000        15  PNC387553A  Person & Covey, Inc.   
9  Oct  6 2022 10:11AM  248000        15  PNC391103A  Person & Covey, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
11  248024       Released         26
12  248027       Released          1
13  248031       Released         12
14  248033      Executing          1
15  248037       Released          8

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
248024                NaN        NaN      26.0
248027                NaN        NaN       1.0
248031                NaN        NaN      12.0
248033                NaN        1.0       NaN
248037                NaN        NaN       8.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
247901                0.0        1.0       0.0
247922                0.0        0.0      44.0
247943                0.0        0.0       1.0
247972                4.0        4.0       6.0
247985                0.0        0.0       3.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
247901                  0          1         0
247922                  0          0        44
247943                  0          0         1
247972                  4          4         6
247985                  0          0         3

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               247901          0          1         0
1               247922          0          0        44
2               247943          0          0         1
3               247972          4          4         6
4               247985          0          0         3

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               247901                   1         
1               247922                           44
2               247943                            1
3               247972         4         4        6
4               247985                            3

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0    Oct  6 2022 10:14AM  248037        15   Person & Covey, Inc.
8    Oct  6 2022 10:11AM  248000        15   Person & Covey, Inc.
28   Oct  6 2022  9:57AM  248033        12              Hush Hush
29   Oct  6 2022  9:47AM  247943        21      NBTY Global, Inc.
30   Oct  6 2022  9:46AM  247901        21      NBTY Global, Inc.
31   Oct  6 2022  9:43AM  248031        10      ISDIN Corporation
43   Oct  6 2022  9:42AM  247922        15  Alliance Pharma, Inc.
87   Oct  6 2022  9:33AM  248024        10      ISDIN Corporation
113  Oct  6 2022  9:31AM  248027        12              SugarBear
114  Oct  6 2022  9:21AM  248021        10      ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Completed  \
0  Oct  6 2022 10:14AM  248037        15   Person & Covey, Inc.             
1  Oct  6 2022 10:11AM  248000        15   Person & Covey, Inc.             
2  Oct  6 2022  9:57AM  248033        12              Hush Hush             
3  Oct  6 2022  9:47AM  247943        21      NBTY Global, Inc.             
4  Oct  6 2022  9:46AM  247901        21      NBTY Global, Inc.             
5  Oct  6 2022  9:43AM  248031        10      ISDIN Corporation             
6  Oct  6 2022  9:42AM  247922        15  Alliance Pharma, Inc.             
7  Oct  6 2022  9:33AM  248024        10      ISDIN Corporation             
8  Oct  6 2022  9:31AM  248027        12              SugarBear             
9  Oct  6 2022  9:21AM  248021        10      ISDIN Corporation             

  Executing Released  
0                  8  
1                 20  
2         1           
3                  1  
4         1           
5                 12  
6                 44  
7                 26  
8                  1  
9                 23

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer Released  \
0  Oct  6 2022 10:14AM  248037        15   Person & Covey, Inc.        8   
1  Oct  6 2022 10:11AM  248000        15   Person & Covey, Inc.       20   
2  Oct  6 2022  9:57AM  248033        12              Hush Hush            
3  Oct  6 2022  9:47AM  247943        21      NBTY Global, Inc.        1   
4  Oct  6 2022  9:46AM  247901        21      NBTY Global, Inc.            
5  Oct  6 2022  9:43AM  248031        10      ISDIN Corporation       12   
6  Oct  6 2022  9:42AM  247922        15  Alliance Pharma, Inc.       44   
7  Oct  6 2022  9:33AM  248024        10      ISDIN Corporation       26   
8  Oct  6 2022  9:31AM  248027        12              SugarBear        1   
9  Oct  6 2022  9:21AM  248021        10      ISDIN Corporation       23   

  Executing Completed  
0                      
1                      
2         1            
3                      
4         1            
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse              Customer Released  \
0  Oct  6 2022 10:14AM  248037        15  Person & Covey, Inc.        8   
1  Oct  6 2022 10:11AM  248000        15  Person & Covey, Inc.       20   
2  Oct  6 2022  9:57AM  248033        12             Hush Hush            
3  Oct  6 2022  9:47AM  247943        21     NBTY Global, Inc.        1   
4  Oct  6 2022  9:46AM  247901        21     NBTY Global, Inc.            

  Executing Completed  
0                      
1                      
2         1            
3                      
4         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse              Customer  Released  \
0  Oct  6 2022 10:14AM  248037        15  Person & Covey, Inc.       8.0   
1  Oct  6 2022 10:11AM  248000        15  Person & Covey, Inc.      20.0   
2  Oct  6 2022  9:57AM  248033        12             Hush Hush       NaN   
3  Oct  6 2022  9:47AM  247943        21     NBTY Global, Inc.       1.0   
4  Oct  6 2022  9:46AM  247901        21     NBTY Global, Inc.       NaN   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        1.0        NaN  
3        NaN        NaN  
4        1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse              Customer  Released  \
0  Oct  6 2022 10:14AM  248037        15  Person & Covey, Inc.       8.0   
1  Oct  6 2022 10:11AM  248000        15  Person & Covey, Inc.      20.0   
2  Oct  6 2022  9:57AM  248033        12             Hush Hush       0.0   
3  Oct  6 2022  9:47AM  247943        21     NBTY Global, Inc.       1.0   
4  Oct  6 2022  9:46AM  247901        21     NBTY Global, Inc.       0.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        1.0        0.0  
3        0.0        0.0  
4        1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1736042      98.0        4.0        4.0
12          496060       1.0        1.0        0.0
15          743959      72.0        0.0        0.0
21          495844       1.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1736042      98.0        4.0        4.0
1        12   496060       1.0        1.0        0.0
2        15   743959      72.0        0.0        0.0
3        21   495844       1.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      98.0        4.0        4.0
1        12       1.0        1.0        0.0
2        15      72.0        0.0        0.0
3        21       1.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released   98.0
1        12   Released    1.0
2        15   Released   72.0
3        21   Released    1.0
4        10  Executing    4.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   21
Status                         
Completed   4.0  0.0   0.0  0.0
Executing   4.0  1.0   0.0  1.0
Released   98.0  1.0  72.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   21
0          Completed   4.0  0.0   0.0  0.0
1          Executing   4.0  1.0   0.0  1.0
2           Released  98.0  1.0  72.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   21
0  Completed   4.0  0.0   0.0  0.0
1  Executing   4.0  1.0   0.0  1.0
2   Released  98.0  1.0  72.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()